In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

MIRNA_PATH = './data/pancanMiRs_EBadjOnProtocolPlatformWithoutRepsWithUnCorrectMiRs_08_04_16.csv'

In [2]:
# Read miRNA data
data_mirna = pd.read_csv(MIRNA_PATH, sep=',')
data_mirna = data_mirna.drop(labels=["Correction"], axis=1)
mirna_samples = pd.read_csv(MIRNA_PATH, sep=',', header=None, nrows=1)
mirna_samples = mirna_samples.values.tolist()
mirna_samples = mirna_samples[0]
mirna_samples = mirna_samples[2:]
for j in range(len(mirna_samples)):
	mirna_samples[j] = mirna_samples[j][:12]
data_mirna.columns = ['sample'] + mirna_samples
data_mirna.fillna(0.0, inplace=True)
data_mirna = data_mirna.T
data_mirna.drop(index='sample', inplace=True)

In [3]:
# Read patients ID in preprocessed clinical data
data_clin = pd.read_csv('./preprocessed_data/Pc_clinical_emb.csv', header=None)
clin_samples = data_clin[[0]]
clin_samples = clin_samples.values.tolist()
clinical_samples = list()
for i in range(len(clin_samples)):
	clinical_samples.append(clin_samples[i][0])
clin_samples = clinical_samples

In [5]:
# Remove the rows with same patient ID
data_mirna.reset_index(inplace=True)
data_mirna = data_mirna.drop_duplicates(['index'])
data_mirna.reset_index(drop=True)
data_mirna.set_index('index', inplace=True)

In [17]:
# min-max normalization
scaler = MinMaxScaler()
mirna_0_1 = scaler.fit_transform(data_mirna)
mirna_f_df = pd.DataFrame(mirna_0_1)
mirna_f_df.index = data_mirna.index
mirna_f_df.reset_index(inplace=True)

In [18]:
# Create all zero vector
a = mirna_f_df[mirna_f_df['index'] == 'TCGA-AG-3586']
sample_row = a.copy()
sample_row['index'] = 'xx'
for j in range(mirna_f_df.shape[1]-1):
	sample_row[j] = 0.0

In [19]:
i = 0
for x in clin_samples:
	if i == 0:
		mirna = mirna_f_df[mirna_f_df['index']  == x]
		if mirna.shape[0] == 0:
			mirna = sample_row.copy()
			mirna['index'] = x
		i += 1
	else:
		mirna_row = mirna_f_df[mirna_f_df['index'] == x]
		if mirna_row.shape[0] == 0:
			mirna_row = sample_row.copy()
			mirna_row['index'] = x
		mirna = pd.concat([mirna, mirna_row], axis = 0)

In [20]:
mirna

,index,0,1,2,3,4,5,6,7,8,...,733,734,735,736,737,738,739,740,741,742
8270,TCGA-OR-A5J1,0.001255,0.082104,0.211161,0.052060,0.038877,0.001522,0.015350,0.034953,0.134515,...,0.000000,0.000000,0.0,0.004835,0.000000,0.000000,0.0,0.000000,0.000000,0.0
8208,TCGA-OR-A5J2,0.005758,0.077737,0.074035,0.050431,0.028982,0.001876,0.012908,0.021848,0.116100,...,0.000449,0.002522,0.0,0.000000,0.002220,0.000000,0.0,0.000000,0.000000,0.0
8219,TCGA-OR-A5J3,0.001673,0.022781,0.115274,0.046547,0.027597,0.000712,0.005815,0.005040,0.034580,...,0.000000,0.000000,0.0,0.012476,0.000000,0.000000,0.0,0.000000,0.000000,0.0
8224,TCGA-OR-A5J4,0.000988,0.055404,0.176683,0.057618,0.037104,0.002467,0.030136,0.038296,0.165027,...,0.000000,0.000000,0.0,0.000000,0.018166,0.000250,0.0,0.000165,0.000944,0.0
8228,TCGA-OR-A5J5,0.000398,0.050076,0.180232,0.024527,0.030323,0.000408,0.003437,0.021120,0.126313,...,0.000000,0.000000,0.0,0.000000,0.007293,0.000000,0.0,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9499,TCGA-YZ-A980,0.001294,0.002486,0.095579,0.010372,0.030969,0.001783,0.026117,0.004487,0.065657,...,0.000000,0.000000,0.0,0.000000,0.008785,0.000503,0.0,0.000267,0.001331,0.0
9503,TCGA-YZ-A982,0.000566,0.013400,0.471902,0.026524,0.138708,0.003975,0.122657,0.022937,0.151600,...,0.000000,0.000000,0.0,0.004815,0.000000,0.000000,0.0,0.000000,0.000000,0.0
9508,TCGA-YZ-A983,0.000398,0.009458,0.260531,0.010492,0.037071,0.003529,0.031463,0.055730,0.122461,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0
9523,TCGA-YZ-A984,0.002698,0.009453,0.372791,0.035748,0.107389,0.006083,0.179279,0.034054,0.241981,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000597,0.000000,0.0


In [20]:
# Save data
mirna.set_index('index', inplace=True)
mirna.to_csv(f'./preprocessed_data/PC_miRNA.csv', header=False, index=False)

*OV Cancer*

In [25]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler


MIRNA_PATH = './data/pancanMiRs_EBadjOnProtocolPlatformWithoutRepsWithUnCorrectMiRs_08_04_16.csv'

In [26]:
# Read miRNA data
data_mirna = pd.read_csv(MIRNA_PATH, sep=',')
data_mirna = data_mirna.drop(labels=["Correction"], axis=1)
mirna_samples = pd.read_csv(MIRNA_PATH, sep=',', header=None, nrows=1)
mirna_samples = mirna_samples.values.tolist()
mirna_samples = mirna_samples[0]
mirna_samples = mirna_samples[2:]
for j in range(len(mirna_samples)):
	mirna_samples[j] = mirna_samples[j][:12]
data_mirna.columns = ['sample'] + mirna_samples
data_mirna.fillna(0.0, inplace=True)
data_mirna = data_mirna.T
data_mirna.drop(index='sample', inplace=True)

In [27]:
# Read patients ID in preprocessed clinical data
data_clin = pd.read_csv('./preprocessed_data/Pc_clinical_emb_OV.csv', header=None)
clin_samples = data_clin[[0]]
clin_samples = clin_samples.values.tolist()
clinical_samples = list()
for i in range(len(clin_samples)):
	clinical_samples.append(clin_samples[i][0])
clin_samples = clinical_samples

In [28]:
data_mirna = data_mirna.loc[data_mirna.index.isin(clin_samples)]

In [29]:
# Remove the rows with same patient ID
data_mirna.reset_index(inplace=True)
data_mirna = data_mirna.drop_duplicates(['index'])
data_mirna.reset_index(drop=True)
data_mirna.set_index('index', inplace=True)

In [30]:
# min-max normalization
scaler = MinMaxScaler()
mirna_0_1 = scaler.fit_transform(data_mirna)
mirna_f_df = pd.DataFrame(mirna_0_1)
mirna_f_df.index = data_mirna.index
mirna_f_df.reset_index(inplace=True)

In [38]:
# Create all zero vector
a = mirna_f_df.loc[mirna_f_df["index"] == mirna_f_df["index"].iloc[-1]]
sample_row = a.copy()
sample_row['index'] = 'xx'
for j in range(mirna_f_df.shape[1]-1):
	sample_row[j] = 0.0

In [41]:
i = 0
data_list = []
for x in clin_samples:
	if i == 0:
		mirna = mirna_f_df[mirna_f_df['index']  == x]
		if mirna.shape[0] == 0:
			mirna = sample_row.copy()
			mirna['index'] = x
		data_list.append(mirna)
		i += 1
	else:
		mirna_row = mirna_f_df[mirna_f_df['index'] == x]
		if mirna_row.shape[0] == 0:
			mirna_row = sample_row.copy()
			mirna_row['index'] = x
		data_list.append(mirna_row)

		# mirna = pd.concat([mirna, mirna_row], axis = 0)

In [46]:
mirna = pd.concat(data_list)

In [47]:
# Save data
mirna.set_index('index', inplace=True)
mirna.to_csv(f'./preprocessed_data/PC_miRNA_OV.csv', header=False, index=False)

In [48]:
(mirna.sum(axis=1) == 0).sum()/ len(mirna)

0.18384879725085912

In [50]:
(mirna.sum(axis=1) == 0).sum()

107

In [51]:
mirna

,0,1,2,3,4,5,6,7,8,9,...,733,734,735,736,737,738,739,740,741,742
index,,,,,,,,,,,,,,,,,,,,,
TCGA-04-1331,0.011621,0.042241,0.624315,0.245888,0.301778,0.054566,0.334344,0.173486,0.447913,0.209873,...,0.000000,0.0,0.0,0.0,0.034956,0.0,0.0,0.0,0.0,0.0
TCGA-04-1332,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
TCGA-04-1335,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
TCGA-04-1336,0.000000,0.040930,0.069092,0.365796,0.207381,0.081868,0.071654,0.384740,0.123701,0.220477,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
TCGA-04-1337,0.062669,0.071184,0.140452,0.183367,0.109292,0.111235,0.063823,0.174663,0.189592,0.311878,...,0.394219,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-61-2614,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
TCGA-OY-A56P,0.022007,0.837398,0.277226,0.010092,0.021171,0.184369,0.089673,0.006510,0.132005,0.032182,...,0.000000,0.0,0.0,0.0,0.012412,0.0,0.0,0.0,0.0,0.0
TCGA-OY-A56Q,0.044804,0.947961,0.143630,0.000000,0.000000,0.183836,0.047159,0.000272,0.019787,0.047690,...,0.033653,0.0,0.0,0.0,0.072413,0.0,0.0,0.0,0.0,0.0
